In [39]:
import os
import shutil
from conlleval import evaluate
from collections import defaultdict
import pandas as pd
from multiprocessing import Pool

In [40]:
for res in ["finest"]:
    f1 = []
    accuracy = []
    labels = defaultdict(list)
    for root, dirs, files in os.walk('/home/nlp/shovalsa/modality/modality_NN/results/decoded_bert'):
        for file in files:
            if res in file:
                print(file)
                with open(os.path.join(root, file), "r") as f:
                    tags = [(line.split()[0], line.split()[1]) for line in f.readlines()]
                    tags = list(map(list, zip(*tags)))
                    true_tags, pred_tags = tags[0], tags[1]
                    true_tags, pred_tags = convert_pred_to_bottom_up(true_tags), convert_pred_to_bottom_up(pred_tags)
                    result = evaluate(true_tags, pred_tags, verbose=True)
                    print(results)
                    f1.append(result['results'][-1])
                    accuracy.append(result["accuracy"])
                    for label, v in result["labels"].items():
                        labels[label].append(v["f1"])
    print(res)
    print("f1: ", sum(f1)/5)
    print("accuracy: ", sum(accuracy)/5)
    for label, l_f1 in labels.items():
        print(label, sum(l_f1)/5)
    print()

finest_fold_2.bmes


NameError: name 'convert_pred_to_top_down' is not defined

In [18]:
# for res in ["binary", "refine", "second_tier", "finest"]:
for res in ["modal_not_modal"]:

    f1 = []
    accuracy = []
    labels = defaultdict(list)
    for root, dirs, files in os.walk('/home/nlp/shovalsa/modality/modality_NN/results/decoded_bert'):
        for file in files:
            if res in file:
                print(file)
                with open(os.path.join(root, file), "r") as f:
                    tags = [(line.split()[0], line.split()[1]) for line in f.readlines()]
                    tags = list(map(list, zip(*tags)))
                    true_tags, pred_tags = tags[0], tags[1]
                    result = evaluate(true_tags, pred_tags, verbose=True)
                    f1.append(result['results'][-1])
                    accuracy.append(result["accuracy"])
                    for label, v in result["labels"].items():
                        labels[label].append(v["f1"])
    print(res)
    print(f1)
    print("f1: ", sum(f1)/5)
    print("accuracy: ", sum(accuracy)/5)
    for label, l_f1 in labels.items():
        print(label, sum(l_f1)/5)
    print()


modal_not_modal_fold_2.bmes
modal_not_modal_fold_1.bmes
modal_not_modal_fold_3.bmes
modal_not_modal_fold_0.bmes
modal_not_modal_fold_4.bmes
modal_not_modal
[71.82781882428526, 69.64285714285715, 69.7209782983121, 68.9893283113622, 69.24778761061947]
f1:  69.88575403748725
accuracy:  65.1530475662117
modal 69.88575403748725



In [41]:
def convert_pred_to_bottom_up(pred_tags):
    converted = []
    for tag in pred_tags:
        if len(tag.strip()) > 1:
            tag = tag.split("-")
            prefix = tag[0]
            suffix = tag[1]
            if suffix in ["deontic", "buletic", "teleological", "intentional", "buletic_teleological"]:
                converted.append(f"{prefix}-priority")
            elif suffix in ["epistemic", "ability", "circumstantial", "opportunity", "ability_circumstantial",
                           "epistemic_circumstantial"]:
                converted.append(f"{prefix}-plausibility")
            else:
                converted.append(f"{prefix}-{suffix}")
        else:
            converted.append(tag)
    return converted

## BASELINE

In [35]:
def get_fold_train_and_dev_sets_by_res(res, fold):
    train_df = pd.read_csv(f"/home/nlp/shovalsa/modality/data/GME/{res}/{fold}/train_{res}_space.bmes", sep=" ", names=["token", "label"])
    dev_df = pd.read_csv(f"/home/nlp/shovalsa/modality/data/GME/{res}/{fold}/dev_{res}_space.bmes", sep=" ",names=["token", "label"])
    return train_df, dev_df


def get_majority_vote(df):
    majority_vote = dict()
    for token in df["token"].unique():
        label = df[df["token"] == token]["label"].value_counts().idxmax()
        majority_vote[token] = label
    return majority_vote

def get_modal_not_modal(df):
    def set_modal_target(modal):
        if modal == "O":
            return modal
        else:
            prefix = modal.split("-")[0]
            return f"{prefix}-modal"
    df["label"] = df["label"].apply(set_modal_target)
    majority_vote = dict()
    for token in df["token"].unique():
        label = df[df["token"] == token]["label"].value_counts().idxmax()
        majority_vote[token] = label
    return majority_vote    

def convert_true_tags_to_binary(true_tags):
    true = []
    for tag in true_tags:
        if tag == "O":
            true.append(tag)
        else:
            prefix = tag.split("-")[0]
            true.append(f"{prefix}-modal")
    return true


def get_test_true_tags(res):
    test_set = pd.read_csv(f"/home/nlp/shovalsa/modality/data/GME/{res}/test_{res}_space.bmes", sep=" ", names=["token", "label"])
    return test_set, [tag for tag in test_set["label"] if tag]

def get_fold_pred_tags(test_set, condition, model):
    return [model[token]  if token in model else 'O' for token in test_set["token"] if token]


In [30]:
def eval_baseline(foldXres, binary=True):
    fold, res = foldXres[0], foldXres[1]
    test_set, true_tags = get_test_true_tags(res)
    if binary:
        true_tags = convert_true_tags_to_binary(true_tags)
    train, dev = get_fold_train_and_dev_sets_by_res(res, fold)
    if binary:
        model = get_modal_not_modal(train)
    else:
        model = get_majority_vote(train)
    pred_tags = get_fold_pred_tags(test_set, res, model)
    print()
#         results = e.evaluate(true_seqs=true_tags, pred_seqs=pred_tags, verbose=True)
    result = evaluate(true_tags, pred_tags, verbose=True)
    f1 = result['results'][-1]
    acc = result["accuracy"]
    print(f"condition {res} fold {fold} f1 {f1} accuracy {acc}")
    for label, v in result["labels"].items():
        label_f1 = v["f1"]
        print(f"condition {res} fold {fold} label {label} f1 {label_f1}")

      
    
    


In [36]:
pool = [(0, "binary"), (1, "binary"),(2, "binary"),(3, "binary"),(4, "binary")]#,
#        (0, "finest"), (1, "finest"),(2, "finest"),(3, "finest"),(4, "finest"),
#        (0, "second_tier"), (1, "second_tier"),(2, "second_tier"),(3, "second_tier"),(4, "second_tier")]

with Pool(5) as p:
    p.map(eval_baseline, pool)


condition binary fold 4 f1 68.34924965893588 accuracy 58.16917728852839
condition binary fold 4 label modal f1 68.34924965893588

condition binary fold 2 f1 68.35616438356163 accuracy 57.93742757821553
condition binary fold 2 label modal f1 68.35616438356163

condition binary fold 1 f1 68.30601092896174 accuracy 57.93742757821553
condition binary fold 1 label modal f1 68.30601092896174

condition binary fold 0 f1 67.35112936344969 accuracy 57.12630359212051
condition binary fold 0 label modal f1 67.35112936344969

condition binary fold 3 f1 68.44262295081968 accuracy 58.16917728852839
condition binary fold 3 label modal f1 68.44262295081968


In [37]:
(68.34924965893588+68.35616438356163+68.30601092896174+67.35112936344969+68.44262295081968)/5

68.16103545714573

In [38]:
results = """
condition finest fold 0 f1 46.06661941885188 accuracy 37.77520278099652
condition finest fold 0 label ability f1 62.60869565217391
condition finest fold 0 label ability_circumstantial f1 16.666666666666668
condition finest fold 0 label buletic f1 0
condition finest fold 0 label buletic_teleological f1 0
condition finest fold 0 label circumstantial f1 52.54901960784314
condition finest fold 0 label deontic f1 53.84615384615385
condition finest fold 0 label epistemic f1 49.61832061068702
condition finest fold 0 label epistemic_circumstantial f1 3.7037037037037033
condition finest fold 0 label priority f1 2.8985507246376807
condition finest fold 0 label teleological f1 46.57534246575342

condition binary fold 0 f1 63.58620689655172 accuracy 53.53418308227115
condition binary fold 0 label plausibility f1 71.96765498652292
condition binary fold 0 label priority f1 54.80225988700565

condition second_tier fold 0 f1 53.04232804232804 accuracy 44.224422442244226
condition second_tier fold 0 label O f1 2.9850746268656714
condition second_tier fold 0 label deontic f1 53.06122448979591
condition second_tier fold 0 label epistemic f1 47.5
condition second_tier fold 0 label intentional f1 41.23711340206185
condition second_tier fold 0 label opportunity f1 68.16326530612244


condition finest fold 1 f1 46.984572230014024 accuracy 38.818076477404404
condition finest fold 1 label ability f1 62.71186440677966
condition finest fold 1 label ability_circumstantial f1 16.666666666666668
condition finest fold 1 label buletic f1 0
condition finest fold 1 label buletic_teleological f1 6.666666666666667
condition finest fold 1 label circumstantial f1 52.96442687747036
condition finest fold 1 label deontic f1 55.187637969094915
condition finest fold 1 label epistemic f1 51.320754716981135
condition finest fold 1 label epistemic_circumstantial f1 3.6363636363636362
condition finest fold 1 label priority f1 0
condition finest fold 1 label teleological f1 45.63758389261745

condition binary fold 1 f1 63.47469220246237 accuracy 53.76593279258401
condition binary fold 1 label plausibility f1 72.09612817089453
condition binary fold 1 label priority f1 54.4179523141655

condition second_tier fold 1 f1 54.11764705882353 accuracy 45.54455445544554
condition second_tier fold 1 label O f1 2.777777777777778
condition second_tier fold 1 label deontic f1 54.382022471910105
condition second_tier fold 1 label epistemic f1 50.455927051671736
condition second_tier fold 1 label intentional f1 41.02564102564102
condition second_tier fold 1 label opportunity f1 69.12065439672801


condition finest fold 2 f1 47.36842105263158 accuracy 38.70220162224797
condition finest fold 2 label ability f1 66.10169491525426
condition finest fold 2 label ability_circumstantial f1 16.000000000000004
condition finest fold 2 label buletic f1 0
condition finest fold 2 label buletic_teleological f1 0
condition finest fold 2 label circumstantial f1 52.38095238095239
condition finest fold 2 label deontic f1 54.99999999999999
condition finest fold 2 label epistemic f1 52.67175572519084
condition finest fold 2 label epistemic_circumstantial f1 4.166666666666667
condition finest fold 2 label priority f1 3.03030303030303
condition finest fold 2 label teleological f1 44.736842105263165

condition binary fold 2 f1 62.758620689655174 accuracy 52.838933951332564
condition binary fold 2 label plausibility f1 70.96774193548387
condition binary fold 2 label priority f1 54.107648725212464

condition second_tier fold 2 f1 53.703703703703695 accuracy 44.77447744774477
condition second_tier fold 2 label O f1 2.73972602739726
condition second_tier fold 2 label deontic f1 54.462242562929056
condition second_tier fold 2 label epistemic f1 50.31055900621118
condition second_tier fold 2 label intentional f1 40.0
condition second_tier fold 2 label opportunity f1 68.45360824742268


condition finest fold 3 f1 46.9187675070028 accuracy 38.818076477404404
condition finest fold 3 label ability f1 63.86554621848739
condition finest fold 3 label ability_circumstantial f1 18.181818181818183
condition finest fold 3 label buletic f1 0
condition finest fold 3 label buletic_teleological f1 0
condition finest fold 3 label circumstantial f1 52.58964143426296
condition finest fold 3 label deontic f1 55.233853006681514
condition finest fold 3 label epistemic f1 52.23880597014925
condition finest fold 3 label epistemic_circumstantial f1 3.7037037037037033
condition finest fold 3 label priority f1 0
condition finest fold 3 label teleological f1 44.736842105263165


condition binary fold 3 f1 64.29061000685401 accuracy 54.461181923522595
condition binary fold 3 label plausibility f1 72.67833109017496
condition binary fold 3 label priority f1 55.58659217877094


condition second_tier fold 3 f1 54.177545691906005 accuracy 45.654565456545654
condition second_tier fold 3 label O f1 2.857142857142857
condition second_tier fold 3 label deontic f1 54.5045045045045
condition second_tier fold 3 label epistemic f1 50.0
condition second_tier fold 3 label intentional f1 40.20100502512563
condition second_tier fold 3 label opportunity f1 69.65376782077392

condition finest fold 4 f1 46.47887323943662 accuracy 38.35457705677868
condition finest fold 4 label ability f1 67.79661016949152
condition finest fold 4 label ability_circumstantial f1 0
condition finest fold 4 label buletic f1 0
condition finest fold 4 label buletic_teleological f1 0
condition finest fold 4 label circumstantial f1 53.28185328185329
condition finest fold 4 label deontic f1 55.233853006681514
condition finest fold 4 label epistemic f1 50.0
condition finest fold 4 label epistemic_circumstantial f1 3.7735849056603774
condition finest fold 4 label priority f1 2.8985507246376807
condition finest fold 4 label teleological f1 40.55944055944056


condition binary fold 4 f1 63.972602739726014 accuracy 54.34530706836617
condition binary fold 4 label plausibility f1 72.92225201072387
condition binary fold 4 label priority f1 54.6218487394958

condition second_tier fold 4 f1 54.199475065616795 accuracy 45.54455445544554
condition second_tier fold 4 label O f1 2.857142857142857
condition second_tier fold 4 label deontic f1 53.96825396825397
condition second_tier fold 4 label epistemic f1 51.05105105105105
condition second_tier fold 4 label intentional f1 39.17525773195876
condition second_tier fold 4 label opportunity f1 69.95884773662551

"""

agg = {
    "binary": {"plausibility": [], "priority": []},
    "second_tier": {"deontic": [], "epistemic": [], "intentional": [], "opportunity": []},
    "finest": {
        "ability": [], 
        "ability_circumstantial": [], 
        "buletic": [], 
        "buletic_teleological": [], 
        "circumstantial": [], 
        "deontic": [], 
        "epistemic": [], 
        "epistemic_circumstantial": [], 
        "priority": [], 
        "teleological": [], 
    }
}

results = [line.split() for line in results.split("\n") if line.strip()]
for resolution, labels in agg.items():
    f1 = []
    for result in results:
        if resolution in result:
            if "label" not in result:
                f1.append(float(result[5]))
            else:
                for label_name, f1s in labels.items():
                    if label_name == result[5]:
                        f1s.append(float(result[-1]))
    print(resolution)
    print("f1: ", sum(f1)/5)
    for label, l_f1 in labels.items():
        print(label, sum(l_f1)/5)
    print()
    

binary
f1:  63.61654650704986
plausibility 72.12642163876004
priority 54.70726036893008

second_tier
f1:  53.84813991247561
deontic 54.07564959947871
epistemic 49.86350742178679
intentional 40.32780343695745
opportunity 69.07002870153451

finest
f1:  46.76345068958739
ability 64.61688227243735
ability_circumstantial 13.503030303030306
buletic 0.0
buletic_teleological 1.3333333333333335
circumstantial 52.75317871647642
deontic 54.90029956572236
epistemic 51.16992740460165
epistemic_circumstantial 3.7968045232196177
priority 1.7654808959156782
teleological 44.44921022566756



In [ ]:
46.066+46.066